In [112]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,f1_score
import collections
from IPython.display import Image
import os

In [113]:
training_df = pd.read_csv(os.path.join(os.path.abspath(""), '../datasets/merged/training_dataset.csv'))
test_df = pd.read_csv(os.path.join(os.path.abspath(""), '../datasets/merged/test_dataset.csv')) 
training_df.drop('Unnamed: 0',axis=1,inplace=True)
num_rows_with_nan = training_df.isna().any(axis=1).sum()

print(f"Number of rows with at least one NaN: {num_rows_with_nan}")
training_df.head()


Number of rows with at least one NaN: 0


,Date_ID,Month,NumberOfDaysInMonth,Quarter,Year,LeapYear,Minimum Temperature,Maximum Temperature,Barley,Canola,...,Quebec,Saskatchewan,Territories,Yukon,Atlantic_Region,British Columbia_Region,Canada_Region,Prairies_Region,Territories_Region,Total_Value
0,88,5,31,2,2008,1,3,29,False,True,...,True,False,False,False,False,False,True,False,False,1
1,32,9,30,3,2003,0,5,30,False,False,...,False,False,False,False,False,False,False,True,False,1
2,35,12,31,4,2003,0,5,34,False,False,...,False,False,False,False,False,True,False,False,False,1
3,94,11,30,4,2008,1,3,29,False,True,...,False,True,False,False,False,False,False,True,False,2
4,62,3,31,1,2006,0,5,34,False,False,...,False,True,False,False,False,False,False,True,False,1


In [114]:
X_train = training_df.drop('Total_Value',axis=1)
X_train.head()

,Date_ID,Month,NumberOfDaysInMonth,Quarter,Year,LeapYear,Minimum Temperature,Maximum Temperature,Barley,Canola,...,Prince Edward Island,Quebec,Saskatchewan,Territories,Yukon,Atlantic_Region,British Columbia_Region,Canada_Region,Prairies_Region,Territories_Region
0,88,5,31,2,2008,1,3,29,False,True,...,False,True,False,False,False,False,False,True,False,False
1,32,9,30,3,2003,0,5,30,False,False,...,False,False,False,False,False,False,False,False,True,False
2,35,12,31,4,2003,0,5,34,False,False,...,False,False,False,False,False,False,True,False,False,False
3,94,11,30,4,2008,1,3,29,False,True,...,False,False,True,False,False,False,False,False,True,False
4,62,3,31,1,2006,0,5,34,False,False,...,False,False,True,False,False,False,False,False,True,False


In [115]:
y_train = training_df['Total_Value']
y_train.head()

0    1
1    1
2    1
3    2
4    1
Name: Total_Value, dtype: int64

In [116]:
X_test = test_df.drop(['Unnamed: 0','Total_Value'],axis=1)
y_test = test_df['Total_Value']

In [117]:
X_test.head()

,Date_ID,Month,NumberOfDaysInMonth,Quarter,Year,LeapYear,Minimum Temperature,Maximum Temperature,Barley,Canola,...,Prince Edward Island,Quebec,Saskatchewan,Territories,Yukon,Atlantic_Region,British Columbia_Region,Canada_Region,Prairies_Region,Territories_Region
0,209,6,30,2,2018,0,5,28,True,False,...,False,False,False,False,False,False,False,True,False,False
1,17,6,30,2,2002,0,3,29,False,True,...,False,False,True,False,False,False,False,False,True,False
2,246,7,31,3,2021,0,5,29,False,False,...,False,False,False,False,False,False,False,False,True,False
3,107,12,31,4,2009,0,3,29,False,True,...,False,False,False,False,False,False,False,False,True,False
4,19,8,31,3,2002,0,5,34,False,False,...,False,False,False,False,False,False,False,False,True,False


In [118]:
y_test.head()

0    1
1    1
2    2
3    2
4    1
Name: Total_Value, dtype: int64

In [119]:
criteria = ["gini","entropy"]
splitters = ["best","random"]
classifiers_scores = []
#print("SPLIT")
for i in range(5,50,5):
    for criterion in criteria:
        for splitter in splitters:
            classifier = DecisionTreeClassifier(criterion=criterion,splitter=splitter,max_depth=i)
            classifier.fit(X_train, y_train)

            
            y_pred = classifier.predict(X_test)
            score = f1_score(y_test, y_pred, average='macro')
            classifiers_scores.append((criterion, splitter, i, "max_depth", score))

            print("Max Depth",str(i),criterion,splitter)
            print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred))

for i in range(5,50,5):
    for criterion in criteria:
        for splitter in splitters:
            classifier = DecisionTreeClassifier(criterion=criterion,splitter=splitter,min_samples_split=i)
            classifier.fit(X_train, y_train)

            
            y_pred = classifier.predict(X_test)
            score = f1_score(y_test, y_pred, average='macro')
            classifiers_scores.append((criterion, splitter, i, "min_samples_split", score))
            

            print("Min Samples",str(i),criterion,splitter)
            print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred))

Max Depth 5 gini best
[[   0    1    0    0    0    0    0    0    0]
 [   0 1046    8    1    0    0    0    0    0]
 [   0   26   35   18    0    0    0    0    0]
 [   0    3    5   23    0    0    0    0    0]
 [   0    0    0   16    0    0    0    0    0]
 [   0    0    0    7    0    0    0    0    0]
 [   0    0    0    3    0    0    0    0    0]
 [   0    0    0    3    0    0    0    0    0]
 [   0    0    0    1    0    0    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.97      0.99      0.98      1055
           2       0.73      0.44      0.55        79
           3       0.32      0.74      0.45        31
           4       0.00      0.00      0.00        16
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1

   

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.97      0.99      0.98      1055
           2       0.72      0.62      0.67        79
           3       0.46      0.42      0.44        31
           4       0.38      0.62      0.48        16
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1

    accuracy                           0.94      1196
   macro avg       0.28      0.30      0.29      1196
weighted avg       0.92      0.94      0.93      1196

Max Depth 10 gini best
[[   0    1    0    0    0    0    0    0    0]
 [   0 1043   11    1    0    0    0    0    0]
 [   0   13   54   11    0    0    1    0    0]
 [   0    3   12   12    4    0    0    0    0]
 [   0    0    1    4   10    1    0    0    0]
 [   0    0    0    0    6    0 

Max Depth 10 entropy best
[[   0    1    0    0    0    0    0    0    0]
 [   2 1041   12    0    0    0    0    0    0]
 [   0   12   59    8    0    0    0    0    0]
 [   0    1    8   19    3    0    0    0    0]
 [   0    0    1    7    8    0    0    0    0]
 [   0    0    0    4    3    0    0    0    0]
 [   0    0    0    2    1    0    0    0    0]
 [   0    0    0    0    3    0    0    0    0]
 [   0    0    0    0    1    0    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.99      0.99      0.99      1055
           2       0.74      0.75      0.74        79
           3       0.47      0.61      0.54        31
           4       0.42      0.50      0.46        16
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         1


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

...

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

Min Samples 40 entropy best
[[   0    1    0    0    0    0    0    0    0]
 [   0 1039   16    0    0    0    0    0    0]
 [   0    8   63    8    0    0    0    0    0]
 [   0    0   10   15    6    0    0    0    0]
 [   0    0    0    4    9    0    3    0    0]
 [   0    0    0    1    4    0    2    0    0]
 [   0    0    0    0    2    0    1    0    0]
 [   0    0    0    0    3    0    0    0    0]
 [   0    0    0    0    0    1    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.99      0.98      0.99      1055
           2       0.71      0.80      0.75        79
           3       0.54      0.48      0.51        31
           4       0.38      0.56      0.45        16
           5       0.00      0.00      0.00         7
           6       0.17      0.33      0.22         3
           7       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

Min Samples 45 entropy best
[[   0    1    0    0    0    0    0    0    0]
 [   0 1039   16    0    0    0    0    0    0]
 [   0    8   66    5    0    0    0    0    0]
 [   0    0   12   13    6    0    0    0    0]
 [   0    0    0    4    9    0    3    0    0]
 [   0    0    0    1    4    0    2    0    0]
 [   0    0    0    0    2    0    1    0    0]
 [   0    0    0    0    3    0    0    0    0]
 [   0    0    0    0    0    1    0    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.99      0.98      0.99      1055
           2       0.70      0.84      0.76        79
           3       0.57      0.42      0.48        31
           4       0.38      0.56      0.45        16
           5       0.00      0.00      0.00         7
           6       0.17      0.33      0.22         3
           7       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: Unde

In [120]:
classifiers_scores.sort(key=lambda x: x[-1], reverse=True)
print(classifiers_scores[0])

# Print the best classifier
print("Best classifier configuration:")
print("Criterion: {}, Splitter: {}, Value: {}, Parameter: {}, F1 Score: {}".format(*classifiers_scores[0]))

('gini', 'random', 20, 'min_samples_split', 0.3522281817879302)
Best classifier configuration:
Criterion: gini, Splitter: random, Value: 20, Parameter: min_samples_split, F1 Score: 0.3522281817879302


In [121]:
features = X_train.columns.tolist()
type = 'max_depth'
if classifiers_scores[0][3] == type:
    dtree = DecisionTreeClassifier(criterion=classifiers_scores[0][0],splitter=classifiers_scores[0][1],max_depth=classifiers_scores[0][2])
else:
    dtree = DecisionTreeClassifier(criterion=classifiers_scores[0][0],splitter=classifiers_scores[0][1],min_samples_split=classifiers_scores[0][2])
dtree.fit(X_train, y_train)
data = tree.export_graphviz(dtree, out_file=None, feature_names=features, node_ids=True)
graph = pydotplus.graph_from_dot_data(data)
graph.set_size('"100,100!"')
